In [1]:
import geopandas as gpd
from shapely.geometry import box

# 1) Charger landcover
landcover = gpd.read_file(
  r"C:\Users\DELL\Desktop\project_fire\data\raw\landcover\Fusion_Harmonisee"
)
print("CRS d'origine :", landcover.crs)

# 2) Reprojeter en mètres (EPSG 3857)
landcover_m = landcover.to_crs(3857)
print("Nouveau CRS :", landcover_m.crs)

# 3) Dissoudre la zone
study_area_m = landcover_m.dissolve().geometry.iloc[0]
print("Bounds zone d'étude :", study_area_m.bounds)


CRS d'origine : EPSG:4326
Nouveau CRS : EPSG:3857
Bounds zone d'étude : (-965570.4827674902, 2150230.388636569, 1333557.2338643195, 4515197.2595917685)


In [2]:
from shapely.geometry import box
import geopandas as gpd
import numpy as np

cell_size = 2000  # 2km

minx, miny, maxx, maxy = study_area_m.bounds

# Grille rapide avec numpy
xs = np.arange(minx, maxx, cell_size)
ys = np.arange(miny, maxy, cell_size)

grid_cells = [
    box(x, y, x + cell_size, y + cell_size)
    for x in xs
    for y in ys
]

grid = gpd.GeoDataFrame(geometry=grid_cells, crs=3857)

# Clip pour garder juste Algérie + Tunisie
grid_clipped = grid[grid.intersects(study_area_m)]

print("Nombre de cellules finales :", len(grid_clipped))

grid_clipped.head()


Nombre de cellules finales : 813369


,geometry
504,"POLYGON ((-963570.483 3158230.389, -963570.483..."
505,"POLYGON ((-963570.483 3160230.389, -963570.483..."
506,"POLYGON ((-963570.483 3162230.389, -963570.483..."
507,"POLYGON ((-963570.483 3164230.389, -963570.483..."
508,"POLYGON ((-963570.483 3166230.389, -963570.483..."


In [3]:
import pandas as pd
import geopandas as gpd

fire = pd.read_csv( "data/processed/fire_2024_reduced_coordinates.csv")

gdf_fire = gpd.GeoDataFrame(
    fire,
    geometry=gpd.points_from_xy(fire.longitude, fire.latitude),
    crs=4326
)

# Reprojection en mètres (même CRS que la grille)
gdf_fire_m = gdf_fire.to_crs(3857)

print("Nombre de points feu :", len(gdf_fire_m))


Nombre de points feu : 14216


In [4]:
# Transformer en GeoDataFrame
gdf_fire = gpd.GeoDataFrame(
    fire,
    geometry=gpd.points_from_xy(fire.longitude, fire.latitude),
    crs=4326
)

# Reprojection en mètres (même CRS que la grille)
gdf_fire_m = gdf_fire.to_crs(3857)

print("Nombre de points feu :", len(gdf_fire_m))


Nombre de points feu : 14216


In [5]:
fire_with_cell = gpd.sjoin(
    gdf_fire_m,
    grid_clipped,
    how="inner",
    predicate="within"
)

print("Points feu affectés à la grille :", len(fire_with_cell))


Points feu affectés à la grille : 14216


In [6]:
fire_unique = fire_with_cell.drop_duplicates(subset="index_right")

fire_unique["fire"] = 1
fire_unique = fire_unique[["geometry", "fire", "index_right"]]

print("Nombre de cellules contenant un feu :", len(fire_unique))


Nombre de cellules contenant un feu : 2941


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
def chunked_non_fire(grid_clipped, fire_unique, chunk_size=8000):

    cells_with_fire = set(fire_unique["index_right"])
    non_fire_chunks = []

    indices = grid_clipped.index.to_list()
    total = len(indices)

    print("Total cellules dans grille :", total)

    for start in range(0, total, chunk_size):
        end = start + chunk_size
        print(f"  Chunk {start} → {end}")

        chunk = grid_clipped.loc[indices[start:end]]

        # Retirer cellules où il y a un feu
        chunk_no_fire = chunk[~chunk.index.isin(cells_with_fire)].copy()

        # Centroid = point non feu
        chunk_no_fire["geometry"] = chunk_no_fire.geometry.centroid
        chunk_no_fire["fire"] = 0

        non_fire_chunks.append(chunk_no_fire)

    # Fusionner les morceaux
    non_fire = gpd.GeoDataFrame(
        pd.concat(non_fire_chunks, ignore_index=True),
        crs=3857
    )

    print("Nombre total de non-feux :", len(non_fire))
    return non_fire


# Exécution
non_fire = chunked_non_fire(grid_clipped, fire_unique, chunk_size=8000)


Total cellules dans grille : 813369
  Chunk 0 → 8000
  Chunk 8000 → 16000
  Chunk 16000 → 24000
  Chunk 24000 → 32000
  Chunk 32000 → 40000
  Chunk 40000 → 48000
  Chunk 48000 → 56000
  Chunk 56000 → 64000
  Chunk 64000 → 72000
  Chunk 72000 → 80000
  Chunk 80000 → 88000
  Chunk 88000 → 96000
  Chunk 96000 → 104000
  Chunk 104000 → 112000
  Chunk 112000 → 120000
  Chunk 120000 → 128000
  Chunk 128000 → 136000
  Chunk 136000 → 144000
  Chunk 144000 → 152000
  Chunk 152000 → 160000
  Chunk 160000 → 168000
  Chunk 168000 → 176000
  Chunk 176000 → 184000
  Chunk 184000 → 192000
  Chunk 192000 → 200000
  Chunk 200000 → 208000
  Chunk 208000 → 216000
  Chunk 216000 → 224000
  Chunk 224000 → 232000
  Chunk 232000 → 240000
  Chunk 240000 → 248000
  Chunk 248000 → 256000
  Chunk 256000 → 264000
  Chunk 264000 → 272000
  Chunk 272000 → 280000
  Chunk 280000 → 288000
  Chunk 288000 → 296000
  Chunk 296000 → 304000
  Chunk 304000 → 312000
  Chunk 312000 → 320000
  Chunk 320000 → 328000
  Chunk 328

In [8]:
final_geo = pd.concat([
    fire_unique[["geometry", "fire"]],
    non_fire[["geometry", "fire"]]
], ignore_index=True)

print("Dataset total :", final_geo.shape)


Dataset total : (813369, 2)


In [9]:
final_geo = final_geo.to_crs(4326)

final_geo["longitude"] = final_geo.geometry.x
final_geo["latitude"] = final_geo.geometry.y

final = final_geo.drop(columns=["geometry"])

print(final.head())
print(final.shape)


   fire  longitude  latitude
0     1    5.53337  35.70751
1     1    6.46961  32.13579
2     1    6.97630  32.35563
3     1    9.39581  28.19791
4     1    9.49323  28.12826
(813369, 3)


In [10]:
import os

output_dir = "data/processed"
os.makedirs(output_dir, exist_ok=True)

final_path = os.path.join(output_dir, "fire_grid_2km_final.csv")
final.to_csv(final_path, index=False)

print(" Dataset final sauvegardé :", final_path)


 Dataset final sauvegardé : data/processed\fire_grid_2km_final.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === 1) Charger le dataset ===
path = r"C:\data/processed\fire_grid_2km_final.csv"
df = pd.read_csv(path)

print("✔ Dataset chargé !")
print(df.head())
print("\nForme :", df.shape)


# === 2) Statistiques Fire / No Fire ===
counts = df["fire"].value_counts()
percent = counts / counts.sum() * 100

print("\n=== Statistiques Fire / No Fire ===")
print(counts)
print("\n(%) :")
print(percent.round(2))


# === 3) Visualisation simple ===
plt.figure(figsize=(6, 8))

plt.scatter(
    df[df["fire"] == 0]["longitude"],
    df[df["fire"] == 0]["latitude"],
    s=3, label="No Fire", alpha=0.4
)

plt.scatter(
    df[df["fire"] == 1]["longitude"],
    df[df["fire"] == 1]["latitude"],
    s=5, label="Fire", alpha=0.7
)

plt.title("Visualisation Feux vs Non Feux")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DELL\\Desktop\\project_fire\\data\\processed\\fire_grid_2km_final.csv'

In [1]:
import geopandas as gpd
from shapely.geometry import box

# 1) Charger landcover
landcover = gpd.read_file(
    r"C:\Users\DELL\Desktop\project_fire\data\raw\landcover\Fusion\landcover_fusion.shp"
)
print("CRS d'origine :", landcover.crs)

# 2) Reprojeter en mètres (EPSG 3857)
landcover_m = landcover.to_crs(3857)
print("Nouveau CRS :", landcover_m.crs)

# 3) Dissoudre la zone
study_area_m = landcover_m.dissolve().geometry.iloc[0]
print("Bounds zone d'étude :", study_area_m.bounds)


CRS d'origine : EPSG:4326
Nouveau CRS : EPSG:3857
Bounds zone d'étude : (-965570.4827674902, 2150230.388636569, 1333557.2338643195, 4515197.2595917685)


In [2]:
from shapely.geometry import box
import geopandas as gpd
import numpy as np

cell_size = 1000  # 1km

minx, miny, maxx, maxy = study_area_m.bounds

# Grille rapide avec numpy
xs = np.arange(minx, maxx, cell_size)
ys = np.arange(miny, maxy, cell_size)

grid_cells = [
    box(x, y, x + cell_size, y + cell_size)
    for x in xs
    for y in ys
]

grid = gpd.GeoDataFrame(geometry=grid_cells, crs=3857)

# Clip pour garder juste Algérie + Tunisie
grid_clipped = grid[grid.intersects(study_area_m)]

print("Nombre de cellules finales :", len(grid_clipped))

grid_clipped.head()


Nombre de cellules finales : 3247226


,geometry
1010,"POLYGON ((-964570.483 3160230.389, -964570.483..."
1011,"POLYGON ((-964570.483 3161230.389, -964570.483..."
1012,"POLYGON ((-964570.483 3162230.389, -964570.483..."
1013,"POLYGON ((-964570.483 3163230.389, -964570.483..."
1014,"POLYGON ((-964570.483 3164230.389, -964570.483..."


In [3]:
import pandas as pd
import geopandas as gpd

fire = pd.read_csv("data/processed/fire_2024_all.csv")

gdf_fire = gpd.GeoDataFrame(
    fire,
    geometry=gpd.points_from_xy(fire.longitude, fire.latitude),
    crs=4326
)

# Reprojection en mètres (même CRS que la grille)
gdf_fire_m = gdf_fire.to_crs(3857)

print("Nombre de points feu :", len(gdf_fire_m))


Nombre de points feu : 90250


In [4]:
import pandas as pd
import geopandas as gpd

# Charger dataset
fire = pd.read_csv("data/processed/fire_2024_all.csv")

# Garder seulement type = 0
fire = fire[fire["type"] == 0].copy()

# Supprimer la colonne type
fire = fire.drop(columns=["type"])

# Transformer en GeoDataFrame
gdf_fire = gpd.GeoDataFrame(
    fire,
    geometry=gpd.points_from_xy(fire.longitude, fire.latitude),
    crs=4326
)

# Reprojection en mètres (même CRS que la grille)
gdf_fire_m = gdf_fire.to_crs(3857)

print("Nombre de points feu :", len(gdf_fire_m))


Nombre de points feu : 14216


In [5]:
fire_with_cell = gpd.sjoin(
    gdf_fire_m,
    grid_clipped,
    how="inner",
    predicate="within"
)

print("Points feu affectés à la grille :", len(fire_with_cell))


Points feu affectés à la grille : 14216


In [6]:
fire_unique = fire_with_cell.drop_duplicates(subset="index_right")

fire_unique["fire"] = 1
fire_unique = fire_unique[["geometry", "fire", "index_right"]]

print("Nombre de cellules contenant un feu :", len(fire_unique))


Nombre de cellules contenant un feu : 3819


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
def chunked_non_fire(grid_clipped, fire_unique, chunk_size=8000):

    cells_with_fire = set(fire_unique["index_right"])
    non_fire_chunks = []

    indices = grid_clipped.index.to_list()
    total = len(indices)

    print("Total cellules dans grille :", total)

    for start in range(0, total, chunk_size):
        end = start + chunk_size
        print(f"  Chunk {start} → {end}")

        chunk = grid_clipped.loc[indices[start:end]]

        # Retirer cellules où il y a un feu
        chunk_no_fire = chunk[~chunk.index.isin(cells_with_fire)].copy()

        # Centroid = point non feu
        chunk_no_fire["geometry"] = chunk_no_fire.geometry.centroid
        chunk_no_fire["fire"] = 0

        non_fire_chunks.append(chunk_no_fire)

    # Fusionner les morceaux
    non_fire = gpd.GeoDataFrame(
        pd.concat(non_fire_chunks, ignore_index=True),
        crs=3857
    )

    print("Nombre total de non-feux :", len(non_fire))
    return non_fire


# Exécution
non_fire = chunked_non_fire(grid_clipped, fire_unique, chunk_size=8000)


Total cellules dans grille : 3247226
  Chunk 0 → 8000
  Chunk 8000 → 16000
  Chunk 16000 → 24000
  Chunk 24000 → 32000
  Chunk 32000 → 40000
  Chunk 40000 → 48000
  Chunk 48000 → 56000
  Chunk 56000 → 64000
  Chunk 64000 → 72000
  Chunk 72000 → 80000
  Chunk 80000 → 88000
  Chunk 88000 → 96000
  Chunk 96000 → 104000
  Chunk 104000 → 112000
  Chunk 112000 → 120000
  Chunk 120000 → 128000
  Chunk 128000 → 136000
  Chunk 136000 → 144000
  Chunk 144000 → 152000
  Chunk 152000 → 160000
  Chunk 160000 → 168000
  Chunk 168000 → 176000
  Chunk 176000 → 184000
  Chunk 184000 → 192000
  Chunk 192000 → 200000
  Chunk 200000 → 208000
  Chunk 208000 → 216000
  Chunk 216000 → 224000
  Chunk 224000 → 232000
  Chunk 232000 → 240000
  Chunk 240000 → 248000
  Chunk 248000 → 256000
  Chunk 256000 → 264000
  Chunk 264000 → 272000
  Chunk 272000 → 280000
  Chunk 280000 → 288000
  Chunk 288000 → 296000
  Chunk 296000 → 304000
  Chunk 304000 → 312000
  Chunk 312000 → 320000
  Chunk 320000 → 328000
  Chunk 32

In [8]:
final_geo = pd.concat([
    fire_unique[["geometry", "fire"]],
    non_fire[["geometry", "fire"]]
], ignore_index=True)

print("Dataset total :", final_geo.shape)


Dataset total : (3247226, 2)


In [9]:
final_geo = final_geo.to_crs(4326)

final_geo["longitude"] = final_geo.geometry.x
final_geo["latitude"] = final_geo.geometry.y

final = final_geo.drop(columns=["geometry"])

print(final.head())
print(final.shape)


   fire  longitude  latitude
0     1    5.53337  35.70751
1     1    6.46961  32.13579
2     1    6.97630  32.35563
3     1    9.39581  28.19791
4     1    9.49323  28.12826
(3247226, 3)
